# COVID-19 MIDS Collaboration 

## Data Sourcing: COVID Tracking Project 

This Jupyter Notebook reads in raw data as csv files from a website and exports them as [pickle files for faster loading](https://medium.com/better-programming/load-fast-load-big-with-compressed-pickles-5f311584507e). 

This code was adapted from a script provided to us by Professor Kevin Crook of the Berkeley MIDS program during our W205 (Data Engineering) class. 

### Data sources

US COVID-19 data (historical, at state level) from the COVID tracking project: https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/


### Set up environment 

In [1]:
# Import packages
import pandas as pd
import numpy as np
import io
import requests
import pickle 

### Retrieve data

#### Cases

In [2]:
# get data at URL - this URL is for the state historical data, updated daily at 4pm ET
cases = requests.get("https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_confirmed_usafacts.csv")

In [3]:
# check HTTP request status
cases.status_code

200

In [4]:
# just show the first 2000 characters, the text is really long otherwise
cases.text[0:2000]

'\ufeffcountyFIPS,County Name,State,stateFIPS,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20\r\n0,Statewide Unallocated,AL,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [5]:
# load into a Pandas dataframe
cases_covid_df = pd.read_csv(io.StringIO(cases.text))

cases_covid_df

,countyFIPS,County Name,State,stateFIPS,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20
0,0,Statewide Unallocated,AL,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1001,Autauga County,AL,1,0,0,0,0,0,0,...,25,25,25,27,28,30,32,33,36,37
2,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,102,103,109,114,117,123,132,143,147,154
3,1005,Barbour County,AL,1,0,0,0,0,0,0,...,14,15,18,20,22,28,29,30,32,33
4,1007,Bibb County,AL,1,0,0,0,0,0,0,...,23,23,26,28,32,32,33,33,34,35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3190,56037,Sweetwater County,WY,56,0,0,0,0,0,0,...,10,10,10,10,10,10,10,10,10,10
3191,56039,Teton County,WY,56,0,0,0,0,0,0,...,59,61,62,62,62,62,63,63,64,64
3192,56041,Uinta County,WY,56,0,0,0,0,0,0,...,4,6,6,6,6,6,6,6,6,6
3193,56043,Washakie County,WY,56,0,0,0,0,0,0,...,5,5,5,5,5,5,5,5,5,5


We can see that the format is not ideal - this is a wide dataset with columns tracking cases for every date. Below we convert the dataframe to a long format.

In [6]:
# get names of date columns
date_cols = list(cases_covid_df.columns)[4:len(list(cases_covid_df.columns)) - 1]

# convert wide to long
long_cases_covid_df = pd.melt(cases_covid_df, 
        id_vars = ["countyFIPS", "County Name", "State", "stateFIPS"], value_vars = date_cols, 
        var_name='date', value_name='cases')

# inspect
long_cases_covid_df

,countyFIPS,County Name,State,stateFIPS,date,cases
0,0,Statewide Unallocated,AL,1,1/22/20,0
1,1001,Autauga County,AL,1,1/22/20,0
2,1003,Baldwin County,AL,1,1/22/20,0
3,1005,Barbour County,AL,1,1/22/20,0
4,1007,Bibb County,AL,1,1/22/20,0
...,...,...,...,...,...,...
300325,56037,Sweetwater County,WY,56,4/24/20,10
300326,56039,Teton County,WY,56,4/24/20,64
300327,56041,Uinta County,WY,56,4/24/20,6
300328,56043,Washakie County,WY,56,4/24/20,5


#### Deaths

In [7]:
# get data at URL - this URL is for the state historical data, updated daily at 4pm ET
deaths = requests.get("https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_deaths_usafacts.csv")

In [8]:
# check HTTP request status 
deaths.status_code

200

In [9]:
# just show the first 2000 characters, the text is really long otherwise
deaths.text[0:2000]

'\ufeffcountyFIPS,County Name,State,stateFIPS,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20\r\n0,Statewide Unallocated,AL,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [10]:
# load into a Pandas dataframe
deaths_covid_df = pd.read_csv(io.StringIO(deaths.text))

deaths_covid_df

,countyFIPS,County Name,State,stateFIPS,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20
0,0,Statewide Unallocated,AL,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1001,Autauga County,AL,1,0,0,0,0,0,0,...,1,1,1,1,1,1,1,2,2,2
2,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,0,0,1,1,1,3,3,3,3,3
3,1005,Barbour County,AL,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1007,Bibb County,AL,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3190,56037,Sweetwater County,WY,56,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3191,56039,Teton County,WY,56,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3192,56041,Uinta County,WY,56,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3193,56043,Washakie County,WY,56,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


We can see that the format is not ideal - this is a wide dataset with columns tracking cases for every date. Below we convert the dataframe to a long format.

In [11]:
# get names of date columns
deaths_date_cols = list(deaths_covid_df.columns)[4:len(list(deaths_covid_df.columns)) - 1]

# convert wide to long
long_deaths_covid_df = pd.melt(deaths_covid_df, 
        id_vars = ["countyFIPS", "County Name", "State", "stateFIPS"], value_vars = date_cols, 
        var_name='date', value_name='deaths')

# inspect
long_deaths_covid_df

,countyFIPS,County Name,State,stateFIPS,date,deaths
0,0,Statewide Unallocated,AL,1,1/22/20,0
1,1001,Autauga County,AL,1,1/22/20,0
2,1003,Baldwin County,AL,1,1/22/20,0
3,1005,Barbour County,AL,1,1/22/20,0
4,1007,Bibb County,AL,1,1/22/20,0
...,...,...,...,...,...,...
300325,56037,Sweetwater County,WY,56,4/24/20,0
300326,56039,Teton County,WY,56,4/24/20,0
300327,56041,Uinta County,WY,56,4/24/20,0
300328,56043,Washakie County,WY,56,4/24/20,0


#### Population

In [12]:
# get data at URL - this URL is for the state historical data, updated daily at 4pm ET
pop = requests.get("https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_county_population_usafacts.csv")


In [13]:
# check HTTP request status 
pop.status_code

200

In [14]:
# just show the first 2000 characters, the text is really long otherwise
pop.text[0:2000]

'ï»¿countyFIPS,County Name,State,population\r\n0,Statewide Unallocated,AL,0\r\n1001,Autauga County,AL,55869\r\n1003,Baldwin County,AL,223234\r\n1005,Barbour County,AL,24686\r\n1007,Bibb County,AL,22394\r\n1009,Blount County,AL,57826\r\n1011,Bullock County,AL,10101\r\n1013,Butler County,AL,19448\r\n1015,Calhoun County,AL,113605\r\n1017,Chambers County,AL,33254\r\n1019,Cherokee County,AL,26196\r\n1021,Chilton County,AL,44428\r\n1023,Choctaw County,AL,12589\r\n1025,Clarke County,AL,23622\r\n1027,Clay County,AL,13235\r\n1029,Cleburne County,AL,14910\r\n1031,Coffee County,AL,52342\r\n1033,Colbert County,AL,55241\r\n1035,Conecuh County,AL,12067\r\n1037,Coosa County,AL,10663\r\n1039,Covington County,AL,37049\r\n1041,Crenshaw County,AL,13772\r\n1043,Cullman County,AL,83768\r\n1045,Dale County,AL,49172\r\n1047,Dallas County,AL,37196\r\n1049,DeKalb County,AL,71513\r\n1051,Elmore County,AL,81209\r\n1053,Escambia County,AL,36633\r\n1055,Etowah County,AL,102268\r\n1057,Fayette County,AL,16302\r\n10

In [15]:
# load into a Pandas dataframe
pop_df = pd.read_csv(io.StringIO(pop.text))

# fix column name error
pop_df.columns = ['countyFIPS', 'County Name', 'State', 'population']

pop_df

,countyFIPS,County Name,State,population
0,0,Statewide Unallocated,AL,0
1,1001,Autauga County,AL,55869
2,1003,Baldwin County,AL,223234
3,1005,Barbour County,AL,24686
4,1007,Bibb County,AL,22394
...,...,...,...,...
3190,56037,Sweetwater County,WY,42343
3191,56039,Teton County,WY,23464
3192,56041,Uinta County,WY,20226
3193,56043,Washakie County,WY,7805


### Merge data

In [16]:
# check shape
long_cases_covid_df.shape

(300330, 6)

In [17]:
# check shape
long_deaths_covid_df.shape

(300330, 6)

In [18]:
# merge 3 datasets
covid_df = long_cases_covid_df.merge(long_deaths_covid_df, 
                                         left_on = ["countyFIPS", "County Name", "State", "stateFIPS", "date"],
                                         right_on = ["countyFIPS", "County Name", "State", "stateFIPS", "date"],
                                         how = 'left').merge(pop_df, 
                                                             left_on = ["countyFIPS", "County Name", "State"], 
                                                             right_on = ["countyFIPS", "County Name", "State"], 
                                                             how = 'left').add_prefix('o_') 
# inspect
covid_df.shape

(300330, 8)

In [19]:
covid_df.head()

,o_countyFIPS,o_County Name,o_State,o_stateFIPS,o_date,o_cases,o_deaths,o_population
0,0,Statewide Unallocated,AL,1,1/22/20,0,0.0,0.0
1,1001,Autauga County,AL,1,1/22/20,0,0.0,55869.0
2,1003,Baldwin County,AL,1,1/22/20,0,0.0,223234.0
3,1005,Barbour County,AL,1,1/22/20,0,0.0,24686.0
4,1007,Bibb County,AL,1,1/22/20,0,0.0,22394.0


### Convert datatypes

#### Count records

In [20]:
# count rows and columns
covid_df.shape

(300330, 8)

#### Check and convert datatypes

We have:

* o_countyFIPS/ stateFIPS: FIPS codes, should be converted from integer to category 
* o_State/ County Name: state & county name; string
* o_date: should be converted from string to date 
* o_cases: integer
* o_deaths/ population: should be converted from float to integer


In [21]:
# check data types 
covid_df.dtypes

o_countyFIPS       int64
o_County Name     object
o_State           object
o_stateFIPS        int64
o_date            object
o_cases            int64
o_deaths         float64
o_population     float64
dtype: object

In [22]:
covid_df.head()

,o_countyFIPS,o_County Name,o_State,o_stateFIPS,o_date,o_cases,o_deaths,o_population
0,0,Statewide Unallocated,AL,1,1/22/20,0,0.0,0.0
1,1001,Autauga County,AL,1,1/22/20,0,0.0,55869.0
2,1003,Baldwin County,AL,1,1/22/20,0,0.0,223234.0
3,1005,Barbour County,AL,1,1/22/20,0,0.0,24686.0
4,1007,Bibb County,AL,1,1/22/20,0,0.0,22394.0


##### String --> Date columns

In [23]:
# create new version of column as a datetime object - with ymd
covid_df["date"] = pd.to_datetime(covid_df["o_date"])

# check conversion 
covid_df[["o_date", "date"]].head()

,o_date,date
0,1/22/20,2020-01-22
1,1/22/20,2020-01-22
2,1/22/20,2020-01-22
3,1/22/20,2020-01-22
4,1/22/20,2020-01-22


##### Integer --> Categorical columns

In [24]:
# identify which columns to convert 
int_to_category = ["o_countyFIPS", "o_stateFIPS"]
# create new column names
int_to_category_new = list(map(lambda x: x.replace('o_', ''), int_to_category))
# add new converted columns
covid_df[int_to_category_new] = covid_df[int_to_category].apply(lambda x: x.astype('category'))
# check conversion
covid_df[int_to_category + int_to_category_new]


,o_countyFIPS,o_stateFIPS,countyFIPS,stateFIPS
0,0,1,0,1
1,1001,1,1001,1
2,1003,1,1003,1
3,1005,1,1005,1
4,1007,1,1007,1
...,...,...,...,...
300325,56037,56,56037,56
300326,56039,56,56039,56
300327,56041,56,56041,56
300328,56043,56,56043,56


In [25]:
# check conversion
covid_df[int_to_category + int_to_category_new].dtypes

o_countyFIPS       int64
o_stateFIPS        int64
countyFIPS      category
stateFIPS       category
dtype: object

##### Float --> Integer columns

In [26]:
# identify which columns to convert 
to_int = list(covid_df.select_dtypes(include = ["float64"]).columns)
# create new column names
converted_to_int = list(map(lambda x: x.replace('o_', ''), to_int))
# add new converted columns
covid_df[converted_to_int] = covid_df[to_int].apply(lambda x: x.astype('Int64'))
# check conversion
covid_df[to_int + converted_to_int].head()


,o_deaths,o_population,deaths,population
0,0.0,0.0,0,0
1,0.0,55869.0,0,55869
2,0.0,223234.0,0,223234
3,0.0,24686.0,0,24686
4,0.0,22394.0,0,22394


In [27]:
# check conversion
covid_df[to_int + converted_to_int].dtypes

o_deaths        float64
o_population    float64
deaths            Int64
population        Int64
dtype: object

### Check missingness

We have a lot of missing data in the coumns that track COVID related things. However, we don't have any missing data in the ID columns (date, state, fips). Given the difficulty with retrieving these data, one would expect at least some missing data in these columns. 

These missingness counts are a further validation that the datatype conversions did not introduce additional NAs. 

In [28]:
covid_df.isnull().sum(axis = 0)

o_countyFIPS        0
o_County Name       0
o_State             0
o_stateFIPS         0
o_date              0
o_cases             0
o_deaths         1504
o_population      188
date                0
countyFIPS          0
stateFIPS           0
deaths           1504
population        188
dtype: int64

### Generate summary statistics

The summary statistics of original and converted columns are another method for validating the column type conversion. We can see that the conversion did not appear to change summary statistics of the data so from now on we will use the converted columns. 

In [29]:
# display summary statistics of original columns
covid_df[covid_df.columns[pd.Series(covid_df.columns).str.startswith('o_')]].describe(include = 'all')

,o_countyFIPS,o_County Name,o_State,o_stateFIPS,o_date,o_cases,o_deaths,o_population
count,300330.000000,300330,300330,300330.000000,300330,300330.000000,298826.000000,3.001420e+05
unique,NaN,1882,51,NaN,94,NaN,NaN,NaN
top,NaN,Statewide Unallocated,TX,NaN,2/19/20,NaN,NaN,NaN
freq,NaN,4700,23970,NaN,3195,NaN,NaN,NaN
mean,29882.221283,NaN,NaN,30.250391,NaN,48.390374,1.955278,1.027314e+05
std,15522.074855,NaN,NaN,15.157639,NaN,686.507938,39.436089,3.309888e+05
min,0.000000,NaN,NaN,1.000000,NaN,0.000000,0.000000,0.000000e+00
25%,18097.000000,NaN,NaN,18.000000,NaN,0.000000,0.000000,1.040500e+04
50%,29123.000000,NaN,NaN,29.000000,NaN,0.000000,0.000000,2.512700e+04
75%,45055.000000,NaN,NaN,45.000000,NaN,1.000000,0.000000,6.691100e+04


In [30]:
# extract converted columns & unconverted hash column
converted_covid_df = covid_df[list(
    map(lambda x: x.replace('o_', '') if (x != 'o_County Name') & (x != 'o_State') & (x != 'o_cases') else x, 
        list(covid_df[covid_df.columns[pd.Series(covid_df.columns).str.startswith('o_')]].columns)))]
         
# show summary statistics
converted_covid_df.describe(include = 'all')

,countyFIPS,o_County Name,o_State,stateFIPS,date,o_cases,deaths,population
count,300330.0,300330,300330,300330.0,300330,300330.000000,298826.000000,3.001420e+05
unique,3146.0,1882,51,51.0,94,NaN,NaN,NaN
top,0.0,Statewide Unallocated,TX,48.0,2020-02-23 00:00:00,NaN,NaN,NaN
freq,4700.0,4700,23970,23970.0,3195,NaN,NaN,NaN
first,NaN,NaN,NaN,NaN,2020-01-22 00:00:00,NaN,NaN,NaN
last,NaN,NaN,NaN,NaN,2020-04-24 00:00:00,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,48.390374,1.955278,1.027314e+05
std,NaN,NaN,NaN,NaN,NaN,686.507938,39.436089,3.309888e+05
min,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000e+00
25%,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,1.040500e+04


#### Check duplicates

There are no perfectly duplicated rows. 

In [31]:
sum(converted_covid_df.duplicated())

0

There are no duplications for the primary keys - as expected, there appears to be one column for every day, for every state (whether identified via state name or fips code).

In [32]:
# check duplication w/ date and state name
converted_covid_df[converted_covid_df.duplicated(subset=['date','o_State', 'o_County Name'], keep=False)]

,countyFIPS,o_County Name,o_State,stateFIPS,date,o_cases,deaths,population


In [33]:
# check duplication w/ date and state fips code
converted_covid_df[converted_covid_df.duplicated(subset=['date','countyFIPS', 'stateFIPS'], keep=False)]

,countyFIPS,o_County Name,o_State,stateFIPS,date,o_cases,deaths,population


#### Recount records

In [34]:
# count rows 
covid_df.shape[0] == converted_covid_df.shape[0]

True

### Pickle data 

In [35]:
pickle.dump(converted_covid_df, open( "../Data_pkl/covid19/usafacts_covid_df.pkl", "wb" ) )
